In [33]:
#import subprocess
#subprocess.run(['jupyter', 'nbconvert', '--to', 'python', 'light_NN.ipynb'])

import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier 

from __future__ import print_function
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers import Dense, Activation, Dropout

import copy

## Preprocessing

In [34]:

train= pd.read_csv("../data/train.csv")
test= pd.read_csv("../data/test.csv")
#train.head(3)

In [35]:
train= pd.read_csv("../data/train.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
test= pd.read_csv("../data/test.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
combine1 = [train]

for train in combine1: 
        train['Salutation'] = train.Name.str.extract(' ([A-Za-z]+).', expand=False) 
for train in combine1: 
        train['Salutation'] = train['Salutation'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
        train['Salutation'] = train['Salutation'].replace('Mlle', 'Miss')
        train['Salutation'] = train['Salutation'].replace('Ms', 'Miss')
        train['Salutation'] = train['Salutation'].replace('Mme', 'Mrs')
        del train['Name']
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5} 

for train in combine1: 
        train['Salutation'] = train['Salutation'].map(Salutation_mapping) 
        train['Salutation'] = train['Salutation'].fillna(0)
        
for train in combine1: 
        train['Ticket_Lett'] = train['Ticket'].apply(lambda x: str(x)[0])
        train['Ticket_Lett'] = train['Ticket_Lett'].apply(lambda x: str(x)) 
        train['Ticket_Lett'] = np.where((train['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), train['Ticket_Lett'], np.where((train['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0')) 
        train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x)) 
        del train['Ticket'] 
train['Ticket_Lett']=train['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3)

for train in combine1: 
    train['Cabin_Lett'] = train['Cabin'].apply(lambda x: str(x)[0]) 
    train['Cabin_Lett'] = train['Cabin_Lett'].apply(lambda x: str(x)) 
    train['Cabin_Lett'] = np.where((train['Cabin_Lett']).isin([ 'F', 'E', 'D', 'C', 'B', 'A']),train['Cabin_Lett'], np.where((train['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0'))
del train['Cabin'] 
train['Cabin_Lett']=train['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1)



In [36]:
#train.loc[train['Age'].isnull()].head()

## Preprocessing　wish NN

In [37]:
train_drop_age = copy.deepcopy(train)

In [38]:
train_drop_age.dropna(inplace=True)

In [39]:
for_age_train = train_drop_age.drop(['Survived'], axis=1).dropna(axis=0)
X_train_age = train_drop_age.drop('Age', axis=1)
y_train_age = train_drop_age['Age']

In [40]:
# create model
tmodel = Sequential()
tmodel.add(Dense(input_dim=X_train_age.shape[1], units=128,
                 kernel_initializer='normal', bias_initializer='zeros'))
tmodel.add(Activation('relu'))

for i in range(0, 8):
    tmodel.add(Dense(units=64, kernel_initializer='normal',
                     bias_initializer='zeros'))
    tmodel.add(Activation('relu'))
    tmodel.add(Dropout(.25))

tmodel.add(Dense(units=1))
tmodel.add(Activation('linear'))

tmodel.compile(loss='mean_squared_error', optimizer='adam')

In [41]:
%%time
tmodel.fit(X_train_age.values, y_train_age.values, epochs=1000, verbose=2)

Epoch 1/1000
 - 0s - loss: 965.2054
Epoch 2/1000
 - 0s - loss: 478.5117
Epoch 3/1000
 - 0s - loss: 469.9578
Epoch 4/1000
 - 0s - loss: 424.3919
Epoch 5/1000
 - 0s - loss: 434.4350
Epoch 6/1000
 - 0s - loss: 397.3172
Epoch 7/1000
 - 0s - loss: 382.0550
Epoch 8/1000
 - 0s - loss: 348.4034
Epoch 9/1000
 - 0s - loss: 371.0055
Epoch 10/1000
 - 0s - loss: 303.1600
Epoch 11/1000
 - 0s - loss: 327.8920
Epoch 12/1000
 - 0s - loss: 299.8366
Epoch 13/1000
 - 0s - loss: 277.9391
Epoch 14/1000
 - 0s - loss: 261.9032
Epoch 15/1000
 - 0s - loss: 260.4031
Epoch 16/1000
 - 0s - loss: 252.4504
Epoch 17/1000
 - 0s - loss: 249.1071
Epoch 18/1000
 - 0s - loss: 240.7201
Epoch 19/1000
 - 0s - loss: 236.7843
Epoch 20/1000
 - 0s - loss: 235.6774
Epoch 21/1000
 - 0s - loss: 247.2094
Epoch 22/1000
 - 0s - loss: 242.1547
Epoch 23/1000
 - 0s - loss: 237.6591
Epoch 24/1000
 - 0s - loss: 230.6485
Epoch 25/1000
 - 0s - loss: 244.0688
Epoch 26/1000
 - 0s - loss: 230.1064
Epoch 27/1000
 - 0s - loss: 228.6381
Epoch 28/1

In [43]:
to_pred = train.loc[train['Age'].isnull()].drop(['Age'], axis=1)

In [44]:
#to_pred.info()

In [45]:
p = tmodel.predict(to_pred.values)

In [46]:
train['Age'].loc[train['Age'].isnull()] = p.reshape(177,)

/home/iwailab-ml/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [47]:
train['Age']=np.round(train['Age'])

In [48]:
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1
for train in combine1:
    train['IsAlone'] = 0
    train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1

In [49]:
train["Embarked"].fillna(train.Embarked.mean(), inplace=True)
train_data = train.values
xs = train_data[:, 2:] # Pclass以降の変数
y  = train_data[:, 1]  # 正解データ


## test_data

In [50]:
test= pd.read_csv("../data/test.csv")
test= pd.read_csv("../data/test.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)

In [51]:
test["Age"].fillna(train.Age.mean(), inplace=True)
test["Fare"].fillna(train.Fare.mean(), inplace=True)

combine = [test]
for test in combine:
    test['Salutation'] = test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
for test in combine:
    test['Salutation'] = test['Salutation'].replace(['Lady', 'Countess','Capt', 'Col',\
         'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    test['Salutation'] = test['Salutation'].replace('Mlle', 'Miss')
    test['Salutation'] = test['Salutation'].replace('Ms', 'Miss')
    test['Salutation'] = test['Salutation'].replace('Mme', 'Mrs')
    del test['Name']
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for test in combine:
    test['Salutation'] = test['Salutation'].map(Salutation_mapping)
    test['Salutation'] = test['Salutation'].fillna(0)

for test in combine:
        test['Ticket_Lett'] = test['Ticket'].apply(lambda x: str(x)[0])
        test['Ticket_Lett'] = test['Ticket_Lett'].apply(lambda x: str(x))
        test['Ticket_Lett'] = np.where((test['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), test['Ticket_Lett'],
                                   np.where((test['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            '0', '0'))
        test['Ticket_Len'] = test['Ticket'].apply(lambda x: len(x))
        del test['Ticket']
test['Ticket_Lett']=test['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3) 

for test in combine:
        test['Cabin_Lett'] = test['Cabin'].apply(lambda x: str(x)[0])
        test['Cabin_Lett'] = test['Cabin_Lett'].apply(lambda x: str(x))
        test['Cabin_Lett'] = np.where((test['Cabin_Lett']).isin(['T', 'H', 'G', 'F', 'E', 'D', 'C', 'B', 'A']),test['Cabin_Lett'],
                                   np.where((test['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            '0','0'))        
        del test['Cabin']
test['Cabin_Lett']=test['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1).replace("G",1) 

test["FamilySize"] = train["SibSp"] + train["Parch"] + 1

for test in combine:
    test['IsAlone'] = 0
    test.loc[test['FamilySize'] == 1, 'IsAlone'] = 1
    
test_data = test.values
xs_test = test_data[:, 1:]

In [52]:
from sklearn.ensemble import RandomForestClassifier

random_forest=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=15,
            min_weight_fraction_leaf=0.0, n_estimators=51, n_jobs=4,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

random_forest.fit(xs, y)
Y_pred = random_forest.predict(xs_test)


import csv
with open("Predict_result_data.csv", "w") as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(["PassengerId", "Survived"])
    for pid, survived in zip(test_data[:,0].astype(int), Y_pred.astype(int)):
        writer.writerow([pid, survived])